# Dependencies and Setup

In [1]:
# make sure to install these packages before running:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import datetime   # handle date times
import re         # regular expression package
from sqlalchemy import create_engine
import requests
from sodapy import Socrata

Select only the columns we are intersted in and select only accidents that happend in the Austin Metro Area, betweeen 2016 and 2019

* ID
* Severity
* Start_Time
* End_Time
* Start_Lat
* Start_Lng
* City
* County
* State
* Zipcode
* Country
* Visibility(mi) 
* Weather_Condition
* Precipitation(in)
* Sunrise_Sunset
* Civil_Twilight


Also create data frame for the metro cities in Texas



In [2]:
# Check if data has been pickled and if so load pickle
if os.path.exists('data/tx_metro_cities.pkl') :
    tx_metro_cities_df = pd.read_pickle('data/tx_metro_cities.pkl',compression='gzip')    
else:
# pickle only the attributes we are interested in, and compress
# pickle is much faster/smaller than csv
    df = pd.read_csv('data/US_Accidents_Dec19.csv',encoding='utf-8')
    columns_of_interest=[ 'ID','Severity','Start_Time','End_Time','Start_Lat', 'Start_Lng', 'City', 
        'County', 'State','Zipcode', 'Country','Visibility(mi)', 
        'Weather_Condition','Precipitation(in)','Sunrise_Sunset','Civil_Twilight'
    ]
    condition=df['State'] =='TX'
    txDF=df[condition][columns_of_interest]
    #create df for selected cities 
    tx_metro_cities =  ['Austin', 'Round Rock', 'Cedar Park',
                 'San Marcos', 'Georgetown', 'Pflugerville',
                 'Hutto', 'Leander'  ]
    condition = txDF['City'].isin(tx_metro_cities)
    tx_metro_cities_df = txDF[condition]    
    tx_metro_cities_df.to_pickle("data/tx_metro_cities.pkl",compression='gzip')
    



In [3]:
# add_datepart from fastai
# Date Feature Engineering
# will split data colum in to corresponding 'Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear'
# 'Hour', 'Minute'
def add_datepart(df, fldname, drop=True,time=False):
    fld = df[fldname]
    attributes = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear']
    if time: attributes = attributes + ['Hour', 'Minute']
    #, 'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if not np.issubdtype(fld.dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    for n in attributes:
        df[targ_pre+n] = getattr(fld.dt,n.lower())
    df[targ_pre+'Elapsed'] = fld.astype(np.int64) // 10**9
    if drop: df.drop(fldname, axis=1, inplace=True)
        
add_datepart(tx_metro_cities_df,'Start_Time',False,True )        

In [4]:
# use name for DOW
def dow(df):
    days=["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
    return days[df]

tx_metro_cities_df['Start_TimeDayofweek'] =tx_metro_cities_df['Start_TimeDayofweek'].apply(dow)

In [5]:
print(f"Number of Columns : {len(tx_metro_cities_df.columns)}")
print(f"Number of Rows : {len(tx_metro_cities_df)}")
print(tx_metro_cities_df.columns)

tx_metro_cities_df.head()


Number of Columns : 25
Number of Rows : 62609
Index(['ID', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng',
       'City', 'County', 'State', 'Zipcode', 'Country', 'Visibility(mi)',
       'Weather_Condition', 'Precipitation(in)', 'Sunrise_Sunset',
       'Civil_Twilight', 'Start_TimeYear', 'Start_TimeMonth', 'Start_TimeWeek',
       'Start_TimeDay', 'Start_TimeDayofweek', 'Start_TimeDayofyear',
       'Start_TimeHour', 'Start_TimeMinute', 'Start_TimeElapsed'],
      dtype='object')


,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,City,County,State,Zipcode,...,Civil_Twilight,Start_TimeYear,Start_TimeMonth,Start_TimeWeek,Start_TimeDay,Start_TimeDayofweek,Start_TimeDayofyear,Start_TimeHour,Start_TimeMinute,Start_TimeElapsed
261007,A-261009,2,2016-11-30 16:03:54,2016-11-30 17:20:00,30.336502,-97.755646,Austin,Travis,TX,78731,...,Day,2016,11,48,30,Wednesday,335,16,3,1480521834
261008,A-261010,2,2016-11-30 16:32:18,2016-11-30 17:47:02,30.328165,-97.694305,Austin,Travis,TX,78752-2826,...,Day,2016,11,48,30,Wednesday,335,16,32,1480523538
261009,A-261011,2,2016-11-30 16:31:45,2016-11-30 17:46:34,30.326077,-97.692307,Austin,Travis,TX,78752,...,Day,2016,11,48,30,Wednesday,335,16,31,1480523505
261026,A-261028,2,2016-11-30 17:11:22,2016-11-30 17:41:09,30.332523,-97.686707,Austin,Travis,TX,78752,...,Day,2016,11,48,30,Wednesday,335,17,11,1480525882
261037,A-261039,2,2016-11-30 17:16:42,2016-11-30 17:46:32,30.292852,-97.747017,Austin,Travis,TX,78705,...,Day,2016,11,48,30,Wednesday,335,17,16,1480526202


# Austin Fatality Data

In [6]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.austintexas.gov", None)


# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.

results = client.get("ergh-7g8p", limit=2000)
results_df_2012 =pd.DataFrame.from_records(results)

# Convert to pandas DataFrame
results = client.get("vggi-9ddh", limit=2000)
results_df_2013 =pd.DataFrame.from_records(results)

results = client.get("gm9p-snyb", limit=2000)
results_df_2014=pd.DataFrame.from_records(results)

results = client.get("p658-umsa", limit=2000)
results_df_2015=pd.DataFrame.from_records(results)

results = client.get("tiqb-wv3c", limit=2000)
results_df_2016=pd.DataFrame.from_records(results)


results = client.get("ijds-pcyq", limit=2000)
results_df_2017=pd.DataFrame.from_records(results)

results = client.get("9jd4-zjmx", limit=2000)
results_df_2018 = pd.DataFrame.from_records(results)



In [7]:
#concatenate 2016-2018 fatality data
results_project= pd.concat([results_df_2016, results_df_2017, results_df_2018])


print(f"Number of Columns : {len(results_project.columns)}")
print(f"Number of Rows : {len(results_project)}")
print(results_project.columns)

results_project.head()


Number of Columns : 30
Number of Rows : 221
Index(['type', 'fatal_crash_number', 'number_of_fatalities', 'case_number',
       'location', 'area', 'date', 'month', 'day', 'hour', 'time', 'related',
       'charge', 'victim', 'speeding', 'ran_red_light_or_stop_sign',
       'dl_status', 'suspected_impairment', 'restraint_or_helmet',
       'type_of_road', 'ftsra', 'coord_x', 'y_coord', 'killed_driver_pass',
       'dl_status_incident', 'restraint_helmet',
       'failure_to_stop_and_render_aid', 'x_coord', 'case_status',
       'restraint_type'],
      dtype='object')


,type,fatal_crash_number,number_of_fatalities,case_number,location,area,date,month,day,hour,...,ftsra,coord_x,y_coord,killed_driver_pass,dl_status_incident,restraint_helmet,failure_to_stop_and_render_aid,x_coord,case_status,restraint_type
0,Pedestrian,1,1,16-0061621,9700 Blk Lake Creek Pkwy,AD,2016-01-06T00:00:00.000,Jan,Wed,23,...,y,-97.793878,30.475234,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pedestrian,2,1,16-0140992,4800 E. Riverside Dr.,HE,2016-01-14T00:00:00.000,Jan,Thu,14,...,n,-97.717471,30.231659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Motor Vehicle,3,1,16-0221932,6300 blk E William Cannon,FR,2016-01-22T00:00:00.000,Jan,Fri,23,...,n,-97.739337,30.170877,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Pedestrian,4,1,16-0301529,12300 N Lamar,ED,2016-01-30T00:00:00.000,Jan,Sat,20,...,n,-97.679087,30.402741,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Motorcycle,5,1,16-0450263,Burleson Rd / McKinney Falls Pkwy,HE,2016-02-14T00:00:00.000,Feb,Sun,2,...,n,-97.698371,30.191989,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Save to SQLlite


In [8]:
engine = create_engine('sqlite:///data/AUSaccidents.db', echo=False)
tx_metro_cities_df.to_sql('austinAccidents',con=engine,if_exists='replace')


In [9]:
results_project.to_sql('austinFatalities',con=engine,if_exists='replace')
